In [44]:
import pandas as pd
label_proteingym_map = {
    'BG_STRSQ_Abate2015-enrichment':'Q59976_STRSQ_Romero_2015',
    'BLAT_ECOLX_Ranganathan2015-2500': 'BLAT_ECOLX_Stiffler_2015',
    'BLAT_ECOLX_Ostermeier2014-linear':'BLAT_ECOLX_Firnberg_2014',
    'BLAT_ECOLX_Palzkill2012-ddG_stat':'BLAT_ECOLX_Deng_2012',
    'BLAT_ECOLX_Tenaillon2013-singles-MIC_score':'BLAT_ECOLX_Jacquier_2013',
    'DLG4_RAT_Ranganathan2012-CRIPT':'DLG4_RAT_McLaughlin_2012', # pdz
    'GAL4_YEAST_Shendure2015-SEL_C_40h':'GAL4_YEAST_Kitzman_2015',
    'HSP82_YEAST_Bolon2016-selection_coefficient': 'HSP82_YEAST_Mishra_2016',
    'KKA2_KLEPN_Mikkelsen2014-Kan18_avg':'KKA2_KLEPN_Melnikov_2014',
    'RL401_YEAST_Bolon2013-selection_coefficient':'RL40A_YEAST_Roscoe_2013',
    'RL401_YEAST_Bolon2014-react_rel':'RL40A_YEAST_Roscoe_2014',
    'YAP1_HUMAN_Fields2012-singles-linear':'YAP1_HUMAN_Araya_2012'
    
    
    
}

    # 'BRCA1_HUMAN_Fields2015-e3': ?
    # 'GFP_AEQVI_Sarkisyan2016': 'GFP_AEQVI_Sarkisyan_2016'
    # 'MTH3_HAEAESTABILIZED_Tawfik2015-Wrel_G17_filtered': 'MTH3_HAEAE_RockahShmuel_2015'
    #''PABP_YEAST_Fields2013-linear':'PABP_YEAST_Melamed_2013'
    # 'POLG_HCVJF_Sun2014-fitness':'POLG_HCVJF_Qi_2014'
    # 'UBE4B_MOUSE_Klevit2013-nscor_log2_ratio':'UBE4B_MOUSE_Starita_2013'

dataset_score = {}
for label_name,proteingym_name in label_proteingym_map.items():
    proteingym_ground_truth_file = f'/work/commons/proteingym/DMS_ProteinGym_substitutions/{proteingym_name}.csv'
    
    models = ['ESM1b','EVmutation','DeepSequence','UniRep_evotuned']
    value_name = ['esm1b_t33_650M_UR50S','prediction_epistatic','evol_indices_ensemble','Unirep_score']
    value_name = dict(zip(models,value_name))
    models_dict = {}
    for model in models:
        proteingym_model_file = f'/work/commons/proteingym/zero_shot_substitutions_scores/{model}/{proteingym_name}.csv'
        proteingym_model = pd.read_csv(proteingym_model_file)
        models_dict[model] = proteingym_model

    label_file = f'/nethome/zli3161/DATA-nash/combining-evolutionary-and-assay-labelled-data/data/learning-protein-fitness-from-evolutionary-and-assay-labelled-data/processed_data/{label_name}/data.csv'
    import pandas as pd
    proteingym_gt_file = pd.read_csv(proteingym_ground_truth_file)
    label = pd.read_csv(label_file)
    scores = dict(zip(models,[[] for i in range(len(models))]))
    for i, row in label.iterrows():
        
        mutant_key = 'mutant'
        # print(row[mutant_key])
        if row[mutant_key] == 'WT':
            continue

        if label_name in ['DLG4_RAT_Ranganathan2012-CRIPT'] and row[mutant_key] not in proteingym_gt_file[mutant_key].values:
            print(row[mutant_key])
            continue
        assert row[mutant_key] in proteingym_gt_file[mutant_key].values
        idx = proteingym_gt_file[proteingym_gt_file[mutant_key] == row[mutant_key]].index[0]
        # check mutated sequence
        assert row[mutant_key] in proteingym_gt_file[mutant_key][idx]
        
        for model in models:
            # print(model)
            proteingym_model = models_dict[model]
            value = value_name[model]
            if model is not 'UniRep_evotuned':
                model_score = proteingym_model[proteingym_model[mutant_key] == row[mutant_key]][value]
            else:
                model_score = proteingym_model[proteingym_model['mutated_sequence'] == proteingym_gt_file['mutated_sequence'][idx]][value]
            scores[model].append(model_score.values[0])
    dataset_score[label_name] = scores

P311A


In [73]:

key = 'BG_STRSQ_Abate2015-enrichment'
model_dir = dict(zip(models,['esm','ev','DeepSequence','eUniRep']))
import os
for model in models:
    print(model)
    print(len(dataset_score[key][model]))
    pll_values = dataset_score[key][model]

    # 生成相应的id
    ids = [f'id_{i}' for i in range(len(pll_values))]

    # 创建DataFrame
    df = pd.DataFrame({'pll': pll_values}, index=ids)

    # 将结果写入CSV文件
    #mkdir -p inference/{key}/esm
    os.makedirs(f'inference/{key}/{model_dir[model]}', exist_ok=True)
    df.to_csv(f'inference/{key}/{model_dir[model]}/pll.csv', index=True)

ESM1b
2633
EVmutation
2633
DeepSequence
2633
UniRep_evotuned
2633


In [3]:
key = 'BLAT_ECOLX_Stiffler_2015'
import pandas as pd
df_ =pd.read_csv(f'/nethome/zli3161/DATA-nash/combining-evolutionary-and-assay-labelled-data/results/{key}/results.csv')

# df_ =pd.read_csv(f'/nethome/zli3161/DATA-nash/combining-evolutionary-and-assay-labelled-data/results/BLAT_ECOLX_Ranganathan2015-2500/results_BLAT_ECOLX_Ranganathan2015-2500.csv')
CORE_PREDICTORS = [
    # 'onehot',
    # 'ev+onehot', 'gesm+onehot', 'eunirep_ll+onehot',
    # 'vae+onehot', 'ev+onehot+ddg', 'gesm+onehot+ddg', 'eunirep_ll+onehot+ddg',
    # 'vae+onehot+ddg'
    # 'onehot',
    'ev+onehot', 
    # 'gesm+onehot', 
    # 'eunirep_ll+onehot',
    # 'vae+onehot', 
]
for model in CORE_PREDICTORS:
    print(model, df_[df_.predictor==model].spearman.mean())

onehot 0.478847619047619
ev+onehot 0.7542238095238095
gesm+onehot 0.4696857142857142
eunirep_ll+onehot 0.4610285714285714
vae+onehot 0.4575952380952381


In [76]:
df_

,dataset,predictor,predictor_params,seed,n_train,ndcg,ndcg_1mut,spearman,spearman_1mut,topk,topk_mean,topk_mean_1mut
0,BLAT_ECOLX_Ranganathan2015-2500,eunirep_ll+onehot,NaN,0,240,0.5828,0.5828,0.5766,0.5766,96,-0.4014,-0.4014
1,BLAT_ECOLX_Ranganathan2015-2500,eunirep_ll+onehot,NaN,1,240,0.7182,0.7182,0.6353,0.6353,96,-0.2466,-0.2466
2,BLAT_ECOLX_Ranganathan2015-2500,eunirep_ll+onehot,NaN,2,240,0.6595,0.6595,0.6056,0.6056,96,-0.3322,-0.3322
3,BLAT_ECOLX_Ranganathan2015-2500,eunirep_ll+onehot,NaN,3,240,0.6309,0.6309,0.6093,0.6093,96,-0.1463,-0.1463
4,BLAT_ECOLX_Ranganathan2015-2500,eunirep_ll+onehot,NaN,4,240,0.6992,0.6992,0.5946,0.5946,96,-0.2826,-0.2826
...,...,...,...,...,...,...,...,...,...,...,...,...
175,BLAT_ECOLX_Ranganathan2015-2500,vae+onehot+ddg,NaN,15,240,0.9058,0.9058,0.8464,0.8464,96,-0.0127,-0.0127
176,BLAT_ECOLX_Ranganathan2015-2500,vae+onehot+ddg,NaN,16,240,0.8933,0.8933,0.8427,0.8427,96,-0.0597,-0.0597
177,BLAT_ECOLX_Ranganathan2015-2500,vae+onehot+ddg,NaN,17,240,0.8791,0.8791,0.8264,0.8264,96,-0.0435,-0.0435
178,BLAT_ECOLX_Ranganathan2015-2500,vae+onehot+ddg,NaN,18,240,0.8954,0.8954,0.8449,0.8449,96,-0.0536,-0.0536
